In [1]:
import pandas as pd

link_dados = "https://osf.io/download/627ky/?view_only=b0158bcde6a64d0da3a977b44b0610ca"

df = pd.read_excel(link_dados)

df.head(2)

,Recommendation,Decision,all_checks_id_unique,all_checks_and_id,All_checks,Finished_check,Comp_check,Time_check,ids_match,Time_calc,...,politics_exp,politics_exp_1_TEXT,US,US_2_TEXT,state,AGE(1),GENDER(2),REGION(4),ETHNICITY(6),STANDARD_HHI(7)
0,1,1,1,1,1,1,1,1,1,0.395833,...,NaN,NaN,1,NaN,48,61,Male,Midwest,White,"Less than $14,999"
1,1,1,1,1,1,1,1,1,1,0.386574,...,NaN,NaN,1,NaN,78,66,Male,South,White,"$25,000 to $29,999"


Estamos interessados em usar as seguites colunas:

1. Recommendation (1 = recomendação da IA é o tratamento padrão, 2 = recomendação da IA não é o tratamento padrão)
2. Decision (1 = médico seguiu a recomendação da IA, 2 = médico não seguiu a recomendação da IA)
3. Reasonable - Em uma escala de 1 a 7 o participante deveria avaliar o quão "razoável" foi a decisão do médico

Vamos converter os valores numéricos para o conteúdo que desejamos:


In [3]:
df["Recommendation"] = df["Recommendation"].replace(
    {
        1: "Tratamento padrão",
        2: "Tratamento alternativo",
    }
)

df["Decision"] = df["Decision"].replace(
    {
        1: "Seguiu IA",
        2: "Não Seguiu IA",
    }
)

Antes de montarmos nosso modelo, como podemos pensar sobre o problema?

Estamos quantificando variáveis quantitativas ou temos a definição de grupos? Quantos e como definimos esses grupos?

In [5]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

Vamos fazer uma regressão incluindo a interação dos dois fatores:

In [8]:
modelo = ols(
    "reasonable ~ Recommendation * Decision",
    data=df
).fit()

modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             reasonable   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.144
Method:                 Least Squares   F-statistic:                     76.70
Date:                Mon, 12 May 2025   Prob (F-statistic):           8.08e-46
Time:                        15:28:01   Log-Likelihood:                -2657.9
No. Observations:                1356   AIC:                             5324.
Df Residuals:                    1352   BIC:                             5345.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=============================================================================================================================
                                                                coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------
Intercept                                                     4.5458      0.102     44.526      0.000       4.345       4.746
Recommendation[T.Tratamento padrão]                          -0.6712      0.141     -4.753      0.000      -0.948      -0.394
Decision[T.Seguiu IA]                                         0.5431      0.137      3.977      0.000       0.275       0.811
Recommendation[T.Tratamento padrão]:Decision[T.Seguiu IA]     1.3554      0.189      7.189      0.000       0.986       1.725
==============================================================================
Omnibus:                       67.575   Durbin-Watson:                   2.045
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               66.999
Skew:                          -0.501   Prob(JB):                     2.83e-15
Kurtosis:                       2.575   Cond. No.                         7.47
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
anova = anova_lm(
    modelo,
    type=3
)
anova

,df,sum_sq,mean_sq,F,PR(>F)
Recommendation,1.0,3.015994,3.015994,1.018892,3.129627e-01
Decision,1.0,525.144669,525.144669,177.409353,4.017243e-38
Recommendation:Decision,1.0,152.978955,152.978955,51.680802,1.076844e-12
Residual,1352.0,4002.018937,2.960073,NaN,NaN
